# Content Based Filtering

 we will explore Content-based recommendation systems and implement a simple version of one using Python and the Pandas library.

In [ ]:
!wget -O moviedataset.zip https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/data/moviedataset.zip
!unzip -o -j moviedataset.zip 

--2021-05-25 20:15:52--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/data/moviedataset.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160301210 (153M) [application/zip]
Saving to: ‘moviedataset.zip’

moviedataset.zip    100%[===================>] 152.88M  26.8MB/s    in 5.7s    

2021-05-25 20:15:58 (26.6 MB/s) - ‘moviedataset.zip’ saved [160301210/160301210]

Archive:  moviedataset.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: README.txt              
  inflating: tags.csv                


# about datasets

In [ ]:
file = open('README.txt','r')

In [ ]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


removing the year feom title and put it in the new column

In [ ]:

movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)

movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)

movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')

movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [ ]:
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [ ]:
moviesWithGenres_df = movies_df.copy()

for index, row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, genre] = 1
#Filling in the NaN values with 0 to show that a movie doesn't have that column's genre
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
moviesWithGenres_df.shape

(34208, 24)

now rating DAtaFrame

In [ ]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


In [ ]:
ratings_df.drop('timestamp', 1,inplace=True)
ratings_df.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


This technique attempts to figure out what a user's favourite aspects of an item is, and then recommends items that present those aspects. In our case, we're going to try to figure out the input's favorite genres from the movies and ratings given.

#user input movies

In [40]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Silver Linings Playbook', 'rating':5},
            {'title':'Mulholland Drive', 'rating':5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Silver Linings Playbook,5.0
5,Mulholland Drive,5.0


In [41]:
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]

inputMovies = pd.merge(inputId, inputMovies)

inputMovies.drop(columns=['genres','year'],inplace=True)

inputMovies

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1968,"Breakfast Club, The",5.0
4,4848,Mulholland Drive,5.0
5,97921,Silver Linings Playbook,5.0


In [42]:
userMovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
293,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",1994,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1885,1968,"Breakfast Club, The","[Comedy, Drama]",1985,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4753,4848,Mulholland Drive,"[Crime, Drama, Film-Noir, Mystery, Thriller]",2001,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
19868,97921,Silver Linings Playbook,"[Comedy, Drama]",2012,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#USER PROFILE MATRIX

In [43]:
userMovies.reset_index(drop=True, inplace=True)

userGenreTable = userMovies.drop(columns=['movieId','title','genres','year'])
userGenreTable

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
userGenreTable.shape

(6, 20)

In [45]:
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
#The user profile
userProfile

Adventure              5.5
Animation              3.5
Children               5.5
Comedy                18.5
Fantasy                5.5
Romance                0.0
Drama                 20.0
Action                 0.0
Crime                 10.0
Thriller              10.0
Horror                 0.0
Mystery                5.0
Sci-Fi                 0.0
IMAX                   0.0
Documentary            0.0
War                    0.0
Musical                0.0
Western                0.0
Film-Noir              5.0
(no genres listed)     0.0
dtype: float64

In [46]:
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])
genreTable.drop(columns=['movieId','title','genres','year'],inplace=True)
genreTable.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
genreTable.shape

(34208, 20)

In [49]:
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())

In [50]:
recommendationTable_df.sort_values(ascending=False,inplace=True)
recommendationTable_df.head()

movieId
5018      0.841808
81132     0.836158
115333    0.779661
31921     0.779661
76153     0.757062
dtype: float64

# recommendation table

In [57]:
rec_table = movies_df[movies_df.movieId.isin(recommendationTable_df.head(25).keys())]
rec_table.reset_index(drop=True,inplace=True)
rec_table

,movieId,title,genres,year
0,1799,Suicide Kings,"[Comedy, Crime, Drama, Mystery, Thriller]",1997
1,4719,Osmosis Jones,"[Action, Animation, Comedy, Crime, Drama, Roma...",2001
2,5018,Motorama,"[Adventure, Comedy, Crime, Drama, Fantasy, Mys...",1991
3,6902,Interstate 60,"[Adventure, Comedy, Drama, Fantasy, Mystery, S...",2002
4,26104,Murder at the Gallop,"[Comedy, Crime, Drama, Mystery, Thriller]",1963
5,27674,11:14,"[Comedy, Crime, Drama, Mystery, Thriller]",2003
6,27781,Svidd Neger,"[Comedy, Crime, Drama, Horror, Mystery, Romanc...",2003
7,27790,Millions,"[Children, Comedy, Crime, Drama, Fantasy]",2004
8,31921,"Seven-Per-Cent Solution, The","[Adventure, Comedy, Crime, Drama, Mystery, Thr...",1976
9,52202,His Kind of Woman,"[Comedy, Crime, Drama, Film-Noir, Thriller]",1951


###content based filtering is over

#now collaborative filtering

As hinted by its alternate name(user-user filtering), this technique uses other users to recommend items to the input user. It attempts to find users that have similar preferences and opinions as the input and then recommends items that they have liked to the input. There are several methods of finding similar users, and the one we will be using here is going to be based on the **Pearson Correlation Function**.

In [59]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Silver Linings Playbook', 'rating':5},
            {'title':'Mulholland Drive', 'rating':5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Silver Linings Playbook,5.0
5,Mulholland Drive,5.0


In [60]:
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]

inputMovies = pd.merge(inputId, inputMovies)

inputMovies = inputMovies.drop('year', 1)

inputMovies

,movieId,title,genres,rating
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",3.5
1,2,Jumanji,"[Adventure, Children, Fantasy]",2.0
2,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",5.0
3,1968,"Breakfast Club, The","[Comedy, Drama]",5.0
4,4848,Mulholland Drive,"[Crime, Drama, Film-Noir, Mystery, Thriller]",5.0
5,97921,Silver Linings Playbook,"[Comedy, Drama]",5.0


In [61]:
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset.head()

,userId,movieId,rating
19,4,296,4.0
441,12,1968,3.0
479,13,2,2.0
681,14,296,2.0
749,15,1,4.0


In [63]:
userSubset.shape

(202107, 3)

In [62]:
userSubsetGroup = userSubset.groupby(['userId'])

In [64]:
userSubsetGroup.get_group(815)  #for example

,userId,movieId,rating
73747,815,1,4.5
73748,815,2,3.0
73922,815,296,5.0
74678,815,1968,4.5
75916,815,4848,2.0
78263,815,97921,3.0


Let's also sort these groups so the users that share the most movies in common with the input have higher priority. This provides a richer recommendation since we won't go through every single user.


In [65]:
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

# Similarity of users to input user(Pearson Correlation Coefficient)

In [67]:
userSubsetGroup = userSubsetGroup[0:100]

In [68]:
pearsonCorrelationDict = {}

for name, group in userSubsetGroup:

    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
  
    nRatings = len(group)
 
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    
    tempRatingList = temp_df['rating'].tolist()
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
    
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0


In [71]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,0.132260,784
1,0.102240,815
2,0.654654,2108
3,0.878310,2596
4,0.809312,2964


Now let's get the top 50 users that are most similar to the input.

In [72]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
69,0.958315,54308
47,0.952661,39595
15,0.952661,13654
14,0.950824,12728
37,0.935414,30252


#### Rating of selected users to all movies

In [74]:
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

,similarityIndex,userId,movieId,rating
0,0.958315,54308,1,3.0
1,0.958315,54308,2,1.5
2,0.958315,54308,3,2.0
3,0.958315,54308,5,2.0
4,0.958315,54308,7,3.0


In [75]:
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,0.958315,54308,1,3.0,2.874945
1,0.958315,54308,2,1.5,1.437472
2,0.958315,54308,3,2.0,1.916630
3,0.958315,54308,5,2.0,1.916630
4,0.958315,54308,7,3.0,2.874945


In [78]:
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,36.524146,134.858257
2,36.524146,95.054356
3,11.017294,24.319341
4,4.981605,12.424598
5,12.842830,27.660449


#Recommendation DataFrame

In [79]:
recommendation_df = pd.DataFrame()
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,3.692304,1
2,2.602507,2
3,2.207379,3
4,2.494095,4
5,2.153766,5


# top 25 movies for you

In [81]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(25)

,weighted average recommendation score,movieId
movieId,,
85270,5.0,85270
67135,5.0,67135
104386,5.0,104386
104390,5.0,104390
3832,5.0,3832
105001,5.0,105001
26431,5.0,26431
105943,5.0,105943
113559,5.0,113559


In [84]:
rec_table = movies_df[movies_df['movieId'].isin(recommendation_df.head(25)['movieId'].tolist())]
rec_table.reset_index(drop=True,inplace=True)
rec_table

,movieId,title,genres,year
0,3832,"Black Sabbath (Tre volti della paura, I)",[Horror],1963
1,4672,"Tall Guy, The","[Comedy, Romance]",1989
2,5963,"Children's Hour, The",[Drama],1961
3,7136,Stolen Kisses (Baisers volés),"[Comedy, Drama, Romance]",1968
4,26431,Opening Night,[Drama],1977
5,26840,Sonatine (Sonachine),"[Action, Comedy, Crime, Drama]",1993
6,27163,Something Is Happening (Kuch Kuch Hota Hai),"[Comedy, Musical, Romance]",1998
7,31309,Rocco and His Brothers (Rocco e i suoi fratelli),"[Crime, Drama]",1960
8,43482,"Bitter Tea of General Yen, The","[Drama, Romance, War]",1933
9,67135,Night Has a Thousand Eyes,"[Drama, Film-Noir]",1948
